# Final results for Enron-email dataset
In this notebook, we run the VI algorithm to generate the results for Enron email dataset in the paper.

## Step 1: Import libraries

In [2]:
from matplotlib import pyplot as plt
import networkx as nx
import pandas as pd
import time
import numpy as np
from datetime import datetime
import torch
from varhawkes.utils import metrics
from tick.hawkes.inference import HawkesADM4
from multiprocessing import Process
import multiprocessing
import bisect

In [3]:
from torch import optim
from varhawkes import models
from varhawkes import posteriors
from varhawkes import priors
from varhawkes import hawkes_model, excitation_kernels
from varhawkes.learners import VariationalInferenceLearner
from experiment.util import LearnerCallback

## Step 2: Read data

In [4]:
min_timestamp = time.mktime(datetime(2000,1,1).timetuple())

Remove the nodes with less than 10 events

In [5]:
df_ts = pd.read_json('enron_dataset_timeseries.json')
df_ts['timestamps'] = df_ts['timestamps'].apply(np.array)
df_ts['timestamps'] = df_ts['timestamps'].apply(np.unique)
df_ts['timestamps'] = df_ts['timestamps'].apply(lambda events_m: events_m[events_m > min_timestamp] - min_timestamp)
df_ts['num_events_m'] = df_ts['timestamps'].apply(len)
df_ts = df_ts[df_ts.num_events_m > 10] 
df_ts = df_ts.reset_index()

print(df_ts.shape)
df_ts.head()

(143, 3)


,index,timestamps,num_events_m
0,allen-p,"[816180.0, 818700.0, 818940.0, 1078200.0, 1082...",501
1,arnold-j,"[5336760.0, 5337000.0, 6535320.0, 7460940.0, 7...",1112
2,arora-h,"[932280.0, 30199440.0, 32030640.0, 33507360.0,...",66
3,badeer-r,"[14145420.0, 14202360.0, 14310660.0, 14382600....",50
4,bailey-s,"[5141940.0, 5660700.0, 5661660.0, 8087640.0, 8...",187


Scale the time to day-scale

In [6]:
events = list(map(np.array, df_ts.timestamps.values))

scale = 1.0 / (3600 * 24)
end_time = max(map(max, events)) * scale

print('end_time:', end_time)
for m, events_m in enumerate(events):
    events_m = np.unique(np.sort(events_m))
    events_m = events_m * scale
    events[m] = events_m
start_time = min(map(min, events))

end_time: 906.9444212962962


Split the data to test & train with proportion 70-30 %

In [7]:
train_thresh = (end_time-start_time) * 0.7 + start_time

In [8]:
def get_events(events, Thr=train_thresh):
    events_train, events_test = list(), list()
    for i, events_i in enumerate(events):
        id_ = bisect.bisect_right(events_i, train_thresh)
        events_train.append(events_i[:id_])
        events_test.append(events_i[id_:]-train_thresh)
    return events_train, events_test

In [9]:
events_train, events_test = get_events(events, Thr=train_thresh)
end_time_train = max([max(nums) for nums in events_train if len(nums)>0])
end_time_test = max([max(nums) for nums in events_test if len(nums)>0])
num_test = sum(map(len,events_test))
print(f'Number of train datapoints: {sum(map(len,events_train))}')
print(f'Number of test datapoints: {num_test}')

Number of train datapoints: 53392
Number of test datapoints: 20902


In [10]:
end_time_train

635.0290972222222

## Find true adjacency

In [11]:
df_main = pd.read_csv('../../../datasets/enron_cleaned_ts_ready.csv', encoding='utf-8', index_col=0)

In [12]:
import ast

# Minimum timestamp of interest
min_timestamp = time.mktime(datetime(2000,1,1).timetuple())

# Remove unnecessary columns
df_main = df_main[['sender_id', 'receiver_id_list', 'timestamp']]
# Remove messages with unknown sender 
df_main = df_main[~df_main.sender_id.isnull()]
# Remove messages prior to Jan 1, 2000
df_main = df_main[df_main.timestamp >= min_timestamp]
# Parse `receiver_id_list` column from str to list
df_main['receiver_id_list'] = df_main['receiver_id_list'].apply(lambda x: ast.literal_eval(x))

In [13]:
from collections import Counter

df_gp = df_main.groupby('sender_id').agg({
    'receiver_id_list': 'sum', 
    'timestamp': lambda x: list(x),
})
df_gp['receiver_id_list'] = df_gp['receiver_id_list'].apply(lambda rlist: [e for e in rlist if e]) # remove None
df_gp['receiver_id_list'] = df_gp['receiver_id_list'].apply(Counter) # Count number of emails sent to each person

df_gp['timestamp'] = df_gp['timestamp'].apply(np.array)
df_gp['timestamp'] = df_gp['timestamp'].apply(np.unique)

df_gp = df_gp[df_gp['timestamp'].apply(len) > 10]
df_gp.head()

,receiver_id_list,timestamp
sender_id,,
allen-p,"{'holst-k': 100, 'grigsby-m': 63, 'lavorato-j'...","[947497380.0, 947499900.0, 947500140.0, 947759..."
arnold-j,"{'arnold-j': 40, 'lavorato-j': 149, 'ward-k': ...","[952017960.0, 952018200.0, 953216520.0, 954142..."
arora-h,"{'arora-h': 2, 'lavorato-j': 2, 'rogers-b': 6,...","[947613480.0, 976880640.0, 978711840.0, 980188..."
badeer-r,"{'motley-m': 10, 'dasovich-j': 11, 'grigsby-m'...","[960826620.0, 960883560.0, 960991860.0, 961063..."
bailey-s,"{'panus-s': 32, 'mann-k': 5, 'shackleton-s': 5...","[951823140.0, 952341900.0, 952342860.0, 954768..."


In [14]:
idx_to_name_mapping = {i: df_ts['index'].loc[i] for i in df_ts.index}
name_to_idx_mapping = {df_ts['index'].loc[i]: i for i in df_ts.index}

In [15]:
n_nodes = df_gp.shape[0]
adjacency_true = np.zeros((n_nodes, n_nodes))

for sender_id, row in df_gp.iterrows():
    i = name_to_idx_mapping[sender_id]
    for receiver_id, count in row['receiver_id_list'].items():
        try:
            j = name_to_idx_mapping[receiver_id]
            adjacency_true[i,j] = count
        except KeyError:
            pass
        
# Normalize
adjacency_true /= adjacency_true.max()

# Remove diagonal
adjacency_true[adjacency_true < 0.05] = 0.0

# Remove diagonal
adjacency_true[np.eye(n_nodes).astype(bool)] = 0.0

---

## Inference with VI-SG

Set the hyper-parameters of the model

In [17]:
M, T = 10, 5
torch.manual_seed(123456789)
np.random.seed(123465789)

Transofrm the data to torch tensor

In [50]:
events_train, events_test = get_events(events, Thr=train_thresh)
end_time_train = max([max(nums) for nums in events_train if len(nums)>0])
end_time_test = max([max(nums) for nums in events_test if len(nums)>0])
events_test = [torch.tensor(events_m, dtype=torch.float64) for events_m in events_test]
events_train = [torch.tensor(events_m, dtype=torch.float64) for events_m in events_train]

#### Train the VI-SG model

In [26]:
hawkes_model_obj = hawkes_model.HawkesModel(excitation_obj, verbose=True)
posterior_obj = posteriors.LogNormalPosterior()
n_params = n_nodes * (n_nodes * M + 1)
C = 1.0 * torch.ones(n_params, dtype=torch.float64)
prior_obj = priors.GaussianLaplacianPrior(n_nodes, n_params, C)
model = models.ModelHawkesVariational(
    model=hawkes_model_obj, posterior=posterior_obj, prior=prior_obj, n_samples=1, n_weights=1, weight_temp=1e0,
)
x = torch.tensor(
    np.hstack((
        np.hstack((  # alpha, the mean of the parameters
            np.random.normal(loc=0.1, scale=0.1, size=n_nodes),
            np.random.normal(loc=0, scale=0.1, size=M * n_nodes ** 2),)),
        np.hstack((  # beta=log(sigma), log of the variance of the parameters    
            np.log(np.clip(np.random.normal(loc=1.0, scale=0.1, size=n_nodes), 1e-1, 2.0)),
            np.log(np.clip(np.random.normal(loc=1.0, scale=0.1, size=M * n_nodes ** 2), 1e-1, 2.0)),))
    )),
    dtype=torch.float64, requires_grad=True
)
callback = LearnerCallback(x.detach(), adjacency_true.ravel(), 0.01)
opt = optim.Adam([x], lr=0.02)
learner = VariationalInferenceLearner(model=model, optimizer=opt, tol=1e-4, max_iter=15e3,
                                      hyperparam_momentum=0.5, hyperparam_interval=100,hyperparam_offset=0)
learner.fit(events_train, end_time_train, x, callback=callback)

Initialize cache 20449/20449     
iter: 15000 | f1-score: 0.07 | relerr: 0.074 | p@20-50-100: 0.00 0.02 0.01 | dx: 9.39e-02     

tensor([-12.4626, -12.6971, -11.3088,  ...,   0.2667,  -0.1214,   0.1084],
       dtype=torch.float64, requires_grad=True)

### Test the model
Cach the test data

In [21]:
excitation_obj = excitation_kernels.MixtureGaussianFilter(M=M, end_time=T, cut_off=1500.0)
model_likelihood_SG = hawkes_model.HawkesModel(excitation_obj,verbose=True)
model_likelihood_SG.set_data(events_test, end_time=end_time_test)

Initialize cache 20449/20449     


### Get the parameters
With mean

In [37]:
alpha, beta = learner.coeffs.data[:n_params], learner.coeffs.data[n_params:]
z = learner.model.posterior.mean(alpha, beta)
mu_vi = z[:n_nodes].cpu()
adj_vi = z[n_nodes:].cpu().reshape(n_nodes, n_nodes, learner.model.model.excitation.M)
test_likelihood = model_likelihood_SG.log_likelihood(mu_vi,adj_vi)
acc = metrics.accuracy(adj_vi.sum(-1).numpy().ravel(), adjacency_true.flatten(), 1e-4)
f1score = metrics.fscore(adj_vi.sum(-1).numpy().ravel(), adjacency_true.flatten(), 1e-4)

print(f'M : {M} | T : {T} | log_likelihood (mean) : {test_likelihood/ num_test}')

M : 10 | T : 5 | log_likelihood (mean) : -0.19274145427525693


In [36]:
### With mode
z = learner.model.posterior.mode(alpha, beta)
mu_vi = z[:n_nodes].cpu()
adj_vi = z[n_nodes:].cpu().reshape(n_nodes, n_nodes, learner.model.model.excitation.M)

test_likelihood = model_likelihood_SG.log_likelihood(mu_vi,adj_vi)
print(f'M : {M} | T : {T} | log_likelihood (mode): {test_likelihood/ num_test}')

M : 10 | T : 5 | log_likelihood (mode): -0.37269375464954696


----

## Inference with VI-EXP

Set the hyper-parameters of the model

In [46]:
M = 1
decay = 20

Cach the testing data

In [40]:
excitation_obj = excitation_kernels.ExponentialKernel(decay=decay, cut_off=1500.0)
model_likelihood = hawkes_model.HawkesModel(excitation_obj,verbose=True)
model_likelihood.set_data(events_test, end_time=end_time_train)

Initialize cache 20449/20449     


Train the model

In [48]:
excitation_obj = excitation_kernels.ExponentialKernel(decay=decay, cut_off=1500.0)
hawkes_model_obj = hawkes_model.HawkesModel(excitation_obj, verbose=True)
posterior_obj = posteriors.LogNormalPosterior()
n_params = n_nodes * (n_nodes * M + 1)
C = 1.0 * torch.ones(n_params, dtype=torch.float64)
prior_obj = priors.GaussianLaplacianPrior(n_nodes, n_params, C)

model = models.ModelHawkesVariational(
    model=hawkes_model_obj, posterior=posterior_obj, prior=prior_obj, n_samples=1, n_weights=1, weight_temp=1e0,
)
x = torch.tensor(
    np.hstack((
        np.hstack((  # alpha, the mean of the parameters
            np.random.normal(loc=0.1, scale=0.1, size=n_nodes),
            np.random.normal(loc=0, scale=0.1, size=M * n_nodes ** 2),)),
        np.hstack((  # beta=log(sigma), log of the variance of the parameters    
            np.log(np.clip(np.random.normal(loc=1.0, scale=0.1, size=n_nodes), 1e-1, 2.0)),
            np.log(np.clip(np.random.normal(loc=1.0, scale=0.1, size=M * n_nodes ** 2), 1e-1, 2.0)),))
    )),
    dtype=torch.float64, requires_grad=True
)

In [51]:
callback = LearnerCallback(x.detach(), adjacency_true.ravel(), 0.05)

opt = optim.Adam([x], lr=0.02)

learner = VariationalInferenceLearner(model=model, optimizer=opt, tol=1e-4, max_iter=2e4,
                                      hyperparam_momentum=0.5, hyperparam_interval=100,hyperparam_offset=0)
learner.fit(events_train, end_time_train, x, callback=callback)

Initialize cache 20449/20449     
iter: 20000 | f1-score: 0.05 | relerr: 0.123 | p@20-50-100: 0.00 0.02 0.02 | dx: 4.58e-02    

tensor([-3.4036, -2.0673, -6.5648,  ..., -0.2367,  0.0326, -1.7203],
       dtype=torch.float64, requires_grad=True)

Testing the model on test data using the mode of approximate posterior

In [54]:
alpha, beta = learner.coeffs.data[:n_params], learner.coeffs.data[n_params:]
z = learner.model.posterior.mode(alpha, beta)
mu_vi = z[:n_nodes].cpu()
adj_vi = z[n_nodes:].cpu().reshape(n_nodes, n_nodes, learner.model.model.excitation.M)

In [57]:
test_likelihood = model_likelihood.log_likelihood(mu_vi,adj_vi)
print(f'Test Likelihood of VI (mode of the posterior): {test_likelihood/num_test}')

Test Likelihood of VI (mode of the posterior): -0.19476225280328086


Testing the model on test data using the mean of approximate posterior

In [58]:
alpha, beta = learner.coeffs.data[:n_params], learner.coeffs.data[n_params:]
z = learner.model.posterior.mean(alpha, beta)
mu_vi = z[:n_nodes].cpu()
adj_vi = z[n_nodes:].cpu().reshape(n_nodes, n_nodes, learner.model.model.excitation.M)

In [59]:
test_likelihood = model_likelihood.log_likelihood(mu_vi,adj_vi)
print(f'Test Likelihood of VI (mean of the posterior): {test_likelihood/num_test}')

Test Likelihood of VI (mean of the posterior): -0.14706493559708186
